In [22]:
cd /content/drive/My Drive/Vista/ODIR-5K

/content/drive/My Drive/Vista/ODIR-5K


In [23]:
#Importar las librerias
import os

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import glob

%matplotlib inline

In [24]:
#Recopilar las variables del excel
dataset = pd.read_excel(open("./data.xlsx", 'rb'), sheet_name='Sheet1')

In [25]:
#Asignar identificadores a las columnas
dataset.columns = ["id", 'edad', "sexo", "imagenIzquierdo", "imagenDerecho", "diagnosticoIzquierdo", "diagnosticoDerecho", "normal",
                  "diabetes", "glaucoma", "catarata", "amd", "hipertension", "miopia", "otro"]

In [26]:
#Importar librerias para crear la CNN
import sys
import os
from tensorflow.python.keras import backend as prev
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D

In [27]:
prev.clear_session()

In [28]:
imageSize = 80

In [29]:
#Declarar hiperparametros del modelo
outputs = 8
epocas=10
numeroImagenes = 32
pasosEntrenamiento = 800
pasosValidacion = 200
filtroConv1 = 16
filtroConv2 = 32
filtroConv3 = 64
tamanoFiltro1 = (4, 4)
tamanoFiltro2 = (3, 3)
tamanoFiltro3 = (2, 2)
tamanoPool = (2, 2)
variacion = 0.02

In [30]:
#Generar el modelo
cnn = Sequential()
cnn.add(Convolution2D(filtroConv1, tamanoFiltro1, padding ="same", input_shape=(imageSize, imageSize, 3), activation='elu'))
cnn.add(MaxPooling2D(pool_size=tamanoPool))

cnn.add(Convolution2D(filtroConv2, tamanoFiltro2, padding ="same"))
cnn.add(MaxPooling2D(pool_size=tamanoPool, strides=(3,3), padding='same'))

cnn.add(Convolution2D(filtroConv3, tamanoFiltro3, padding ="same"))
cnn.add(MaxPooling2D(pool_size=tamanoPool, strides=(2,2), padding='same'))

cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(outputs, activation='softmax'))

cnn.compile(loss='categorical_crossentropy',
          optimizer=Adam(lr=variacion),
          metrics=['accuracy'])

In [18]:
cnn.layers

In [31]:
#Ojo normal
def normalEye(text):
    if 'normal' in text:
        return 1
    else:
        return 0

In [32]:
#Agregar dos columnas mas.. con las retinas que son normales
dataset['normalIzquierdo'] = dataset['diagnosticoIzquierdo'].apply(lambda diagnostico: normalEye(diagnostico))
dataset['normalDerecho'] = dataset['diagnosticoDerecho'].apply(lambda diagnostico: normalEye(diagnostico))

In [33]:
#Clasificar en listas las imagenes de las retinas
#Miopia
miopiaIzquierdoList = dataset.loc[(dataset.miopia==1) & (dataset.normalIzquierdo==0)]['imagenIzquierdo'].values
miopiaDerechoList = dataset.loc[(dataset.miopia==1) & (dataset.normalDerecho==0)]['imagenDerecho'].values

#Catarata
catarataIzquierdoList = dataset.loc[(dataset.catarata==1) & (dataset.normalIzquierdo==0)]['imagenIzquierdo'].values
catarataDerechoList = dataset.loc[(dataset.catarata==1) & (dataset.normalDerecho==0)]['imagenDerecho'].values

#Diabetes
diabetesIzquierdoList = dataset.loc[(dataset.diabetes==1) & (dataset.normalIzquierdo==0)]['imagenIzquierdo'].values
diabetesDerechoList = dataset.loc[(dataset.diabetes==1) & (dataset.normalDerecho==0)]['imagenDerecho'].values

#Glaucoma
glaucomaIzquierdoList = dataset.loc[(dataset.glaucoma==1) & (dataset.normalIzquierdo==0)]['imagenIzquierdo'].values
glaucomaDerechoList = dataset.loc[(dataset.glaucoma==1) & (dataset.normalDerecho==0)]['imagenDerecho'].values

#AMD
amdIzquierdoList = dataset.loc[(dataset.amd==1) & (dataset.normalIzquierdo==0)]['imagenIzquierdo'].values
amdDerechoList = dataset.loc[(dataset.amd==1) & (dataset.normalDerecho==0)]['imagenDerecho'].values

#Hipertension
hipertensionIzquierdoList = dataset.loc[(dataset.hipertension==1) & (dataset.normalIzquierdo==0)]['imagenIzquierdo'].values
hipertensionDerechoList = dataset.loc[(dataset.hipertension==1) & (dataset.normalDerecho==0)]['imagenDerecho'].values

#Otras enfermedades
otroIzquierdoList = dataset.loc[(dataset.otro==1) & (dataset.normalIzquierdo==0)]['imagenIzquierdo'].values
otroDerechoList = dataset.loc[(dataset.otro==1) & (dataset.normalDerecho==0)]['imagenDerecho'].values

#Normal
normalIzquierdoList = dataset.loc[(dataset.normal==1) & (dataset.normalIzquierdo==1)]['imagenIzquierdo'].values
normalDerechoList = dataset.loc[(dataset.normal==1) & (dataset.normalDerecho==1)]['imagenDerecho'].values

In [34]:
#Unir las listas de cada ojo con el mismo diagnostico
miopiaList = np.concatenate((miopiaIzquierdoList, miopiaDerechoList), axis = 0)
catarataList = np.concatenate((catarataIzquierdoList, catarataDerechoList), axis = 0)
diabetesList = np.concatenate((diabetesIzquierdoList, diabetesDerechoList), axis = 0)
glaucomaList = np.concatenate((glaucomaIzquierdoList, glaucomaDerechoList), axis = 0)
amdList = np.concatenate((amdIzquierdoList, amdDerechoList), axis = 0)
hipertensionList = np.concatenate((hipertensionIzquierdoList, hipertensionDerechoList), axis = 0)
otroList = np.concatenate((otroIzquierdoList, otroDerechoList), axis = 0)
normalList = np.concatenate((normalIzquierdoList, normalDerechoList), axis = 0)

In [30]:
print(len(miopiaList[:206]))
print(len(catarataList))
print(len(diabetesList))
print(len(glaucomaList))
print(len(amdList))
print(len(hipertensionList))
print(len(otroList))
print(len(normalList))

206
367
1995
385
306
206
1482
2275


In [35]:
#Especificar la ruta relativa a la ubicacion actual.. en donde se encuentran las imagenes
imagesTrainPath = "./Training Images"
imagesTestPath = "./Testing images"

In [36]:
#Describir las 8 salidas
from enum import Enum

class Output(Enum):
  miopia       = [1,0,0,0,0,0,0,0]
  catarata     = [0,1,0,0,0,0,0,0]
  diabetes     = [0,0,1,0,0,0,0,0]
  glaucoma     = [0,0,0,1,0,0,0,0]
  amd          = [0,0,0,0,1,0,0,0]
  hipertension = [0,0,0,0,0,1,0,0]
  otro         = [0,0,0,0,0,0,1,0]
  normal       = [0,0,0,0,0,0,0,1]

In [37]:
#Librerias para trabajar con las imagenes
from PIL import Image, ImageOps
from random import shuffle
from tqdm import tqdm
from numpy import asarray

In [38]:
#Funcion para unir etiqueta con ruta
def joinPathLabel(pathList, label):
  resultList = []
  for imageName in tqdm(pathList):
    pathImage = imagesTrainPath+'/'+imageName
    img = Image.open(pathImage)
    imgResized = ImageOps.fit(img, (imageSize,imageSize), Image.ANTIALIAS)
    resultList.append([asarray(imgResized),np.array(label)])
  shuffle(resultList)
  return resultList

In [39]:
miopiaDataSet =joinPathLabel(miopiaList[:206],Output.miopia.value)
catarataDataSet =joinPathLabel(catarataList[:206],Output.catarata.value)
diabetesDataSet =joinPathLabel(diabetesList[:206],Output.diabetes.value)
glaucomaDataSet =joinPathLabel(glaucomaList[:206],Output.glaucoma.value)
amdDataSet =joinPathLabel(amdList[:206],Output.amd.value)
hipertensionDataSet =joinPathLabel(hipertensionList[:206],Output.hipertension.value)
otroDataSet = joinPathLabel(otroList[:206],Output.otro.value)
normalDataSet = joinPathLabel(normalList[:206],Output.normal.value)

100%|██████████| 206/206 [01:24<00:00,  2.44it/s]


In [40]:
trainDataSet = miopiaDataSet + catarataDataSet + diabetesDataSet + glaucomaDataSet + amdDataSet + hipertensionDataSet + otroDataSet + normalDataSet

In [42]:
inputsModel = np.array([i[0] for i in trainDataSet]).reshape(-1,imageSize,imageSize,3)
outputsModel = np.array([i[1] for i in trainDataSet])

In [43]:
from sklearn.model_selection import train_test_split
input_train, input_test, output_train, output_test = train_test_split(inputsModel, outputsModel, test_size=0.2, random_state=42,stratify=outputsModel)

In [ ]:
cnn.fit(input_train, output_train,
                  batch_size=numeroImagenes,
                  epochs=epocas,
                  verbose=1,
                  validation_data=(input_test, output_test))

In [45]:
target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modeloFinal.h5')
cnn.save_weights('./modelo/pesosFinal.h5')

**Correr el modelo**

In [50]:
#Importar librerias para ejecutarlo
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import load_img, array_to_img


In [64]:
longitud, altura = 80, 80
modelo = './modelo/modeloFinal.h5'
pesos_modelo = './modelo/pesosFinal.h5'
cnn2 = tf.keras.models.load_model(modelo)
cnn2.load_weights(pesos_modelo)

In [76]:
def clasificaRetina(x):
  #Cargar imagen
  x = np.expand_dims(x, axis=0)
  array = cnn2.predict(x)
  resultado = array[0]
  print(resultado)
  resp = np.argmax(resultado)
  
  if resp == 0:
    print("Respuesta = Miopia")
  elif resp == 1:
    print("Respuesta = Catarata")
  elif resp == 2:
    print("Respuesta = Diabetes")
  elif resp == 3:
    print("Respuesta = Glaucoma")
  elif resp == 4:
    print("Respuesta = AMD")
  elif resp == 5:
    print("Respuesta = Hipertension")
  elif resp == 6:
    print("Respuesta = Otro")
  elif resp == 7:
    print("Respuesta = Normal")

In [75]:
clasificaRetina(input_test[30])

Respuesta = Miopia


In [72]:
output_test[30]

array([1, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
  # miopia       = [1,0,0,0,0,0,0,0]
  # catarata     = [0,1,0,0,0,0,0,0]
  # diabetes     = [0,0,1,0,0,0,0,0]
  # glaucoma     = [0,0,0,1,0,0,0,0]
  # amd          = [0,0,0,0,1,0,0,0]
  # hipertension = [0,0,0,0,0,1,0,0]
  # otro         = [0,0,0,0,0,0,1,0]
  # normal       = [0,0,0,0,0,0,0,1]